## Lectura como siempre de los dataframes

In [2]:
import pandas as pd


In [3]:
df_primigenio = pd.read_parquet('df_bdev.parquet')
df_ugenre = pd.read_parquet('df_ugenre.parquet')
df_bdev = pd.read_parquet('df_bdev.parquet')
df_sentimiento = pd.read_parquet('df_sentimiento.parquet')

Creación de las funciones que van pidiendo 

Función def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

In [3]:
def UserForGenre(genero: str):
    # Filtrar el DataFrame por el género específico
    df_genre = df_ugenre[df_ugenre['genres'] == genero]

    # Agrupar por usuario y año, sumar las horas jugadas
    grouped_df = df_genre.groupby(['user_id', 'year_release'])['playtime_forever'].sum().reset_index()

    # Encontrar al usuario con más horas jugadas
    max_user = grouped_df.loc[grouped_df['playtime_forever'].idxmax()]['user_id']

    # Filtrar el DataFrame original para obtener las horas jugadas por año del usuario máximo
    max_user_df = df_ugenre[df_ugenre['user_id'] == max_user]
    max_user_hours_by_year = max_user_df.groupby('year_release')['playtime_forever'].sum().reset_index()

    # Crear la lista de la acumulación de horas jugadas por año
    horas_jugadas_por_anio = [{"Año": int(anio), "Horas": int(horas)} for anio, horas in zip(max_user_hours_by_year['year_release'], max_user_hours_by_year['playtime_forever'])]

    # Crear el diccionario de retorno
    resultado = {"Usuario con más horas jugadas para Género " + genero: max_user, "Horas jugadas": horas_jugadas_por_anio}

    return resultado


# Llamada de ejemplo con el género 'Action'
resultado_ejemplo = UserForGenre('Casual')
print(resultado_ejemplo)

{'Usuario con más horas jugadas para Género Casual': '76561198034784208', 'Horas jugadas': [{'Año': 2016, 'Horas': 335}]}


Función def best_developer_year( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

In [4]:
def best_developer_year(year: int):
    # Verificar que las columnas necesarias existan en el DataFrame
    required_columns = {'year_review', 'recommend', 'Sentiment_Score', 'app_name'}
    if not required_columns.issubset(df_bdev.columns):
        raise KeyError(f"Faltan las siguientes columnas necesarias: {required_columns - set(df_bdev.columns)}")

    # Eliminar filas donde 'year_review' es nulo
    df_bdev_cleaned = df_bdev.dropna(subset=['year_review'])

    # Filtrar por año
    df_filtered = df_bdev_cleaned[df_bdev_cleaned['year_review'] == year]

    # Filtrar por recomendaciones positivas/neutrales
    df_filtered = df_filtered[(df_filtered['recommend'] == True) & (df_filtered['Sentiment_Score'].isin([1, 2]))]

    # Obtener el top 3 de juegos más recomendados
    top_games = df_filtered.groupby('app_name')['recommend'].count().sort_values(ascending=False).head(3)

    # Crear el diccionario de resultados en el formato deseado
    top_games_dict = {f'Puesto {i+1}': juego for i, juego in enumerate(top_games.index)}

    # Devolver el diccionario en el formato deseado
    return top_games_dict

# Llamada de ejemplo con el año 2012
resultado_ejemplo1 = best_developer_year(2012)
print(resultado_ejemplo1)

{'Puesto 1': 'Team Fortress 2', 'Puesto 2': 'Terraria', 'Puesto 3': "Garry's Mod"}


Función def developer_reviews_analysis( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

In [6]:
def developer_reviews_analysis(developer):
    # Filtrar el dataframe por la desarrolladora especificada
    developer_df = df_sentimiento[df_sentimiento['developer'] == developer]

    # Sumar los valores de Sentiment_Score para cada categoría
    negative_sum = developer_df[developer_df['Sentiment_Score'] == 0].shape[0]
    neutral_sum = developer_df[developer_df['Sentiment_Score'] == 1].shape[0]
    positive_sum = developer_df[developer_df['Sentiment_Score'] == 2].shape[0]

    # Crear el diccionario de salida
    result_dict = {
        developer: {
            'Negative': negative_sum,
            'Neutral': neutral_sum,
            'Positive': positive_sum
        }
    }

    return result_dict

# Llamada de ejemplo con el año 2012
resultado_ejemplo1 = developer_reviews_analysis('valve')
print(resultado_ejemplo1)

{'valve': {'Negative': 992, 'Neutral': 4851, 'Positive': 3729}}


Función def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [8]:
def userdata(user_id: str):
    # Filtrar el DataFrame para obtener solo los datos del usuario especificado
    user_data = df_bdev[df_bdev['user_id'] == user_id]

    # Calcular el dinero gastado por el usuario
    dinero_gastado = user_data['price'].sum()

    # Calcular el porcentaje de recomendación
    total_reviews = len(user_data)
    recomendaciones_positivas = user_data['recommend'].sum()
    porcentaje_recomendacion = (recomendaciones_positivas / total_reviews) * 100 if total_reviews > 0 else 0

    # Calcular la cantidad de items
    cantidad_items = user_data['item_id'].nunique()

    # Crear el diccionario de resultados
    resultado = {
        "Usuario X": user_id,
        "Dinero gastado": f"{dinero_gastado:.2f} USD",
        "% de recomendación": f"{porcentaje_recomendacion:.2f}%",
        "cantidad de items": cantidad_items
    }

    # Devolver el diccionario en el formato deseado
    return resultado

resultado_ejemplo1 = userdata('76561197970982479')
print(resultado_ejemplo1)


{'Usuario X': '76561197970982479', 'Dinero gastado': '29.98 USD', '% de recomendación': '100.00%', 'cantidad de items': 2}


Función def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de retorno:

In [9]:
def developer(desarrollador: str):
    # Filtrar el DataFrame para obtener solo los datos del desarrollador especificado
    developer_data = df_bdev[df_bdev['developer'].str.lower() == desarrollador.lower()]

    # Verificar si el DataFrame filtrado no está vacío
    if developer_data.empty:
        return f"No se encontraron datos para el desarrollador: {desarrollador}"

    # Agrupar por año y calcular la cantidad de items y el porcentaje de contenido gratuito
    grouped_data = developer_data.groupby('year_release').agg(
        cantidad_items=('item_id', 'count'),
        contenido_free_pct=('price', lambda x: (x == 0).mean() * 100)
    ).reset_index()

    # Renombrar las columnas para el formato deseado
    grouped_data.columns = ['Año', 'Cantidad de Items', 'Contenido Free']

    # Formatear el porcentaje de contenido gratuito
    grouped_data['Contenido Free'] = grouped_data['Contenido Free'].map('{:.2f}%'.format)

    # Devolver el DataFrame con el resultado
    return grouped_data



resultado_ejemplo1 = developer('Stainless Games Ltd')
print(resultado_ejemplo1)

    Año  Cantidad de Items Contenido Free
0  1997                  1          0.00%
1  2011                  2          0.00%
2  2015                 28        100.00%


Función def recomendacion_usuario( id de usuario ): Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

In [10]:
def recomendacion_usuario(user_id: str):
    # Verificar que el usuario existe en el DataFrame
    if user_id not in df_bdev['user_id'].values:
        return f"No se encontró el usuario con id: {user_id}"

    # Obtener los datos de los juegos revisados por el usuario
    juegos_usuario = df_bdev[df_bdev['user_id'] == user_id]

    # Obtener los géneros de los juegos revisados por el usuario
    generos_usuario = juegos_usuario['genres'].unique()

    # Filtrar los juegos que coincidan con los géneros y que el usuario no haya revisado
    juegos_recomendados = df_bdev[(df_bdev['genres'].isin(generos_usuario)) & 
                                  (~df_bdev['item_id'].isin(juegos_usuario['item_id']))]

    # Ordenar los juegos por el número de recomendaciones positivas
    juegos_recomendados['positive_recommendations'] = juegos_recomendados.groupby('item_id')['recommend'].transform('sum')
    top_recomendados = juegos_recomendados.sort_values(by='positive_recommendations', ascending=False).drop_duplicates('item_id').head(5)

    # Crear la lista de juegos recomendados
    lista_recomendaciones = top_recomendados['app_name'].tolist()

    return lista_recomendaciones

resultado_ejemplo1 = recomendacion_usuario('76561197970982479')
print(resultado_ejemplo1)


['Counter-Strike: Global Offensive', 'Left 4 Dead 2', 'BioShock Infinite', 'Counter-Strike: Source', 'Half-Life 2']


C:\Users\User\AppData\Local\Temp\ipykernel_15756\167750673.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  juegos_recomendados['positive_recommendations'] = juegos_recomendados.groupby('item_id')['recommend'].transform('sum')
